# 혼공머 05-3
> 앙상블 (랜덤포레스트 , 엑스트라 트리)

- toc:true
- branch: master
- badges: true
- comments: true
- author: 김동준
- categories : ["Python", "혼공머"]

# 주로쓰는 패키지들

In [31]:
#collapse
import numpy as np #넘파이
import pandas as pd #판다스
from plotnine import *  #플롯나인
import matplotlib.pyplot as plt #맷플랏립
import plotly.express as px #플랏리 상호작용 그래프
from IPython.display import HTML #블로그에 html로 올리려고 변환하는 패키지
import seaborn as sns # 씨본, 히스토그램 깔끔하게 그리는 패키지
#___________________________________

from sklearn.neighbors import KNeighborsClassifier # k 최근접이웃
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor  # 결정계수 
from sklearn.metrics import mean_absolute_error # 타깃과 예측의 절댓값 오차 평균을 반환
from sklearn.linear_model import LinearRegression # 선형 회귀
from sklearn.preprocessing import PolynomialFeatures #다중회귀로의 변환기
from sklearn.preprocessing import StandardScaler #규제
from sklearn.linear_model import Ridge #릿지
from sklearn.linear_model import Lasso #라쏘
from sklearn.linear_model import LogisticRegression # 로지스틱회귀
from scipy.special import expit #시그모이드함수
from scipy.special import softmax #소프트맥스함수
from sklearn.linear_model import SGDClassifier # 확률적 경사 하강법
from sklearn.tree import DecisionTreeClassifier # 트리
from sklearn.tree import plot_tree # 트리 모형
from sklearn.model_selection import cross_validate # 교차 검증
from sklearn.model_selection import StratifiedKFold # Kfold 교차 검증
from sklearn.model_selection import GridSearchCV # 그리드 서치 (하이퍼 파라미터 튜닝)
from scipy.stats import uniform, randint #랜덤 서치
from sklearn.model_selection import RandomizedSearchCV # 랜덤 서치 클래스
from sklearn.ensemble import RandomForestClassifier # 랜덤포레스트 앙상블
from sklearn.ensemble import ExtraTreesClassifier # 엑스트라 트리 앙상블
from sklearn.ensemble import GradientBoostingClassifier # 그레이디언트 부스팅 앙상블

# 히스토그램 기반 그레이디언트 부스팅
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.inspection import permutation_importance # 특성중요도
from xgboost import XGBClassifier # 알고리즘을 구현한 또다른 라이브러리1
from lightgbm import LGBMClassifier # 알고리즘을 구현한 또다른 라이브러리2 , 마이크로소프트에서 구현 


# ____________________


# 정형 데이터와 비정형 데이터
> 정형 데이터 : 어떤 구조(주로 숫자)로 되어 있다는 뜻. CSV ,DB , EXCEL 등에 저장하기 쉽다

> 비정형 데이터 : 텍스트 데이터 , 디카 사진 등 

## 앙상블 학습 (ensemble learning)
> 정형 데이터를 다루는 데 가장 뛰어나 성과를 내는 알고리즘 

# 랜덤 포레스트

> 앙상블 학습의 대표주자

> 결정트리를 랜덤하게 만들어 결저 트리(나무)의 숲을 만듦

> 각 트리를 훈련하기 위한 데이터를 랜덤하게 만드는데, 입력한 훈련 데이터에서 랜덤하게 샘플을 추출하여 훈련 데이터를 만든다. ( 한 샘플이 중복되어 추출 될 수 있음. / 부스트랩 샘플이라고 부름)

> 부스트랩 샘플은 훈련 세트와 크기가 같다. ( 부스트랩 샘플에 포함되지 않고 남는 샘플 : OOB, out of bag)

> 부스트랩 방식 : 데이터 세트에서 중복을 허용하여 데이터를 샘플링하는 방식을 의미.

> RandomForestClassifier 는 자체적으로 모델을 평가하는 정수를 얻을 수 있다.

In [2]:
wine = pd.read_csv('https://bit.ly/wine_csv_data')

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

train_input, test_input, train_target, test_target = \
train_test_split(data, target, test_size=0.2, random_state=42)

In [4]:
# 교차 검증 + 랜덤포레스트
rf = RandomForestClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


In [5]:
# 앞의 랜덤 포레스트 모델을 훈련 세트에 훈련한 후 특성 중요도를 출력
rf.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


> 결정 트리에서 보다 더 골고루 중요도가 분포되었다.

> 과대적합을 줄이고 일반화 성능을 높이는데 기여

In [6]:
# OOB 점수 확인
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)
rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


> OOB 점수는 교차검증을 대신 할 수 있다.

# 엑스트라 트리

> 기본적으로 100개의 결정 트리를 훈련

> 붓스트랩 샘플을 사용하지 않는다.

> 각 결정트리를 만들 떄 전체 훈련 세트를 사용

> 노드 분할시 무작위로 분할

> 빠른 계산 속도

In [8]:
et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


> 랜덤포레스트와 비슷한 결과

In [9]:
# 특성 중요도
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


> 랜덤포레스트와 비슷한 결과

# 그레이디언트 부스팅 , gradient boosting

> 깊이가 얕은 결정 트리를 사용하여 이전 트리의 오차를 보완하는 방식으로 앙상블

> GradientBoostingClassifier은 기본적으로 깊이가 3인 결정 트리를 100개 사용

> 과대적합에 강하고 일반적으로 높은 일반화 성능 기대

> 경사 하강법을 사용하여 트리를 앙상블에 추가

> 분류에서는 로지스틱 손실 함수를 사용 하고 회귀에서는 평균 제곱 오차 함수 사용

In [12]:
# 교차 검증 점수 확인
gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


> 과대적합이 거의 되지 않음

> 트리의 개수를 늘려도 과대적합에 매우 강력

In [13]:
# 학습률을 증가시키고 트리의 개수를 늘리면 조금 더 성능이 향상
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


In [14]:
# 특성 중요도
gb.fit(train_input, train_target)
print(gb.feature_importances_)

[0.15872278 0.68010884 0.16116839]


# 히스토그램 기반 그레이디언트 부스팅

> 입력 특성을 256개의 구간으로 나눔. $\to$ 노드를 분할할 때 최적의 분할을 매우 바르게 찾을 수 있음

> 입력에 누락된 특성이 있어도 이를 따로 전처리할 필요가 없다.

> 아직 테스트 과정이라 추가 모듈 임포트 필요

In [19]:
# 교차 검증 점수 확인
hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9321723946453317 0.8801241948619236


In [21]:
# 특성 중요도
hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target, n_repeats=10,
                                random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]


In [22]:
# 테스트 세트에서의 특성 중요도
result = permutation_importance(hgb, test_input, test_target, n_repeats=10,
                                random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


In [23]:
# 테스트 세트에서의 성능 최종 확인
hgb.score(test_input, test_target)

0.8723076923076923

## 알고리즘을 구현한 또다른 라이브러리1

In [27]:
xgb = XGBClassifier(tree_method='hist', random_state=42)
scores = cross_validate(xgb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9555033709953124 0.8799326275264677


## 알고리즘을 구현한 또다른 라이브러리2

In [30]:
lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.935828414851749 0.8801251203079884


_________

# **마무리**

`*` 키워드로 끝내는 핵심 포인트

`-` **앙상블 학습** : 더 좋은 예측 결과를 만들기 위해 여러 개의 모델을 훈련하는 머신러닝 알고리즘을 말합니다.

`-` **랜덤 포레스트** : 대표적인 결정 트리 기반의 앙상블 학습 방법입니다. 부트스트랩 샘플을 사용하고 랜덤하게 일부 특성을 선택하여 트리를 만드는 것이 특징입니다.

`-` **엑스트라 트리** : 랜덤 포레스트와 비슷하게 결정 트리를 사용하여 앙상블 모델을 만들지만 부트스트랩 샘플을 사용하지 않습니다. 대신 랜덤하게 노드를 분할해 과대적합을 감소시킵니다.

`-` **그레이디언트 부스팅** : 랜덤 포레스트나 엑스트라 트리와 달리 결정 트리를 연속적으로 추가하여 손실 함수를 최소화하는 앙상블 방법입니다. 이런 이유로 훈련 속도가 조금 느리지만 더 좋은 성능을 기대할 수 있습니다. 그레이디언트 부스팅의 속도를 개선한 것이 히스토그램 기반 그레이디언트 부스팅이며 안정적인 결과와 높은 성능으로 매우 인기가 높습니다.

`*` **핵심 패키지**

`*` scikit-learn

> `1` RandomForestClassifier : 랜덤 포레스트 분류 클래스입니다.

>>`-`n_esrimators 매개변수는 앙상블을 구성할 트리의 개수를 지정합니다. 기본값은 100입니다.

>>`-`criterion 매개변수는 불순도를 지정하며 기본값은 지니 불순도를 의미하는 'gini'이고 'entropy'를 선택하여 엔트로피 불순도를 사용할 수 있습니다.

>>`-`max_depth는 트리가 성장할 최대 깊이를 지정합니다. 기본값은 None으로 지정하면 리프노드가 순수하거나 min_samples_split보다 샘플 개수가 적을 때까지 성장합니다.

>>`-`min_samples_split은 노드를 나누기 위한 최소 샘플 개수입니다. 기본값은 2입니다.

>>`-`max_features 매개변수는 최적의 분할을 위해 탐색할 특성의 개수를 지정합니다. 기본값은 auto로 특성 개수의 제곱근입니다.

>>`-`bootstrap 매개변수는 부트스트랩 샘플을 사용할지 지정합니다. 기본값은 True입니다.

>>`-`ooh_score는 OOB 샘플을 사용하여 훈련한 모델을 평가할지 지정합니다. 기본값은 False입니다.

>>`-`n_jobs 매개변수는 병렬 실행에 사용할 CPU 코어 수를 지정합니다. 기본값은 1로 하나의 코어를 사용합니다. -1로 지정하면 시스템에 있는 모든 코어를 사용합니다.

> `2` ExtraTreesClassifier : 엑스트라 트리 분류 클래스입니다.

>>`-`n_estimators, criterion, max_depth, min_samples_split, max_features 매개변수는 랜덤 포레스트와 동일합니다.

>>`-`bootstrap 매개변수는 부트스트랩 샘플을 사용할지 지정합니다. 기본값은 False입니다.

>>`-`oob_score는 OOB 샘플을 사용하여 훈련한 모델을 평가할지 지정합니다. 기본값은 False입니다.

>>`-`n_jobs 매개변수는 병렬 실행에 사용할 CPU 코어 수를 지정합니다. 기본값은 1로 하나의 코어를 사용합니다. -1로 지정하면 시스템에 있는 모든 코어를 사용합니다.

> `3` GradientBoostingClassifier : 그레이디언트 부스팅 분류 클래스입니다.

>>`-`loss 매개변수는 손실 함수를 지정합니다. 기본값은 로지스틱 손실 함수를 의미하는 deviance' 입니다.

>>`-`learning_rate 매개변수는 트리가 앙상블에 기여하는 정도를 조절합니다. 기본값은 0.1입니다.

>>`-`n_estinators 매개변수는 부스팅 단계를 수행하는 트리의 개수입니다. 기본값은 100입니다.

>>`-`subsample 매개변수는 사용할 훈련 세트의 샘플 비율을 지정합니다. 기본값은 1.0입니다.

>>`-`max_depth 매개변수는 개별 회귀 트리의 최대 깊이입니다. 기본값은 3입니다.

> `4` HistGradientBoostingClassifier : 히스토그램 기반 그레이디언트 부스팅 분류 클래스입니다.

>>`-`learning_rate 매개변수는 학습률 또는 감쇠율이라고 합니다. 기본값은 0.1이며 1.0이면 감쇠가 전혀 없습니다.

>>`-`max_iter는 부스팅 단계를 수행하는 트리의 개수입니다. 기본값은 100입니다.

>>`-`max_bins는 입력 데이터를 나눌 구간의 개수입니다. 기본값은 255이며 이보다 크게 지정할 수 없습니다. 여기에 1개의 구간이 누락된 값을 위해 추가됩니다.